This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [1]:
import pandas as pd
import os
import geocoder
import requests

In [2]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [3]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [4]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 5173 N 35TH ST, Milwaukee, WI - [43.11140483819031, -87.9564576503258]
Parsed: 5928 N 39TH ST, Milwaukee, WI - [43.125452161809676, -87.96091583524746]
Parsed: 4235 N 17TH ST, Milwaukee, WI - [43.09467244463039, -87.93192860646873]
Parsed: 3000 W CAPITOL DR, Milwaukee, WI - [43.0899075337582, -87.95081216472387]
Parsed: 4032 N 12TH ST, Milwaukee, WI - [43.09019857507647, -87.92577444460173]
Parsed: 6180 N TEUTONIA AV, Milwaukee, WI - [43.12936099491562, -87.9521672837547]
Parsed: 2222 W CAPITOL DR, Milwaukee, WI - [43.08981144753335, -87.94030110857977]
Parsed: 2000 W ROOSEVELT DR, Milwaukee, WI - [43.09782200279864, -87.9371658475098]
Parsed: 4951 N SHERMAN BL, Milwaukee, WI - [43.107615199001685, -87.96659907372015]
Parsed: 6024 N 42ND ST, Milwaukee, WI - [43.12704290105549, -87.96469504041275]
Parsed: 4356 N 25TH ST, Milwaukee, WI - [43.096354575624716, -87.9440838122224]
Parsed: 6852 N 42ND ST, Milwaukee, WI - [43.142444800998334, -87.96437590074464]
Parsed: 5935-A N 36TH S

Parsed: 3300 S 39TH ST, Milwaukee, WI - [42.98392741326677, -87.96300245340255]
Parsed: 3355 S 27TH ST, Milwaukee, WI - [42.98489091248832, -87.94843751889785]
Parsed: 2339 S 43RD ST, Milwaukee, WI - [43.00193722009348, -87.96776859045453]
Parsed: 1931 S 14TH ST, Milwaukee, WI - [43.009326366639726, -87.92963157372014]
Parsed: 1922 S 8TH ST, Milwaukee, WI - [43.00958713627446, -87.921336407958]
Parsed: 2557 S 12TH ST, Milwaukee, WI - [42.997905031363615, -87.92735007372015]
Parsed: 1725 S 12TH ST, Milwaukee, WI - [43.01168734836635, -87.92701084836635]
Parsed: 539 W VIRGINIA ST, Milwaukee, WI - [43.02632648845881, -87.91798066763806]
Parsed: 539 W VIRGINIA ST, Milwaukee, WI - [43.02632648845881, -87.91798066763806]
Parsed: 818 S WATER ST, Milwaukee, WI - [43.0222021306118, -87.90711142535162]
Parsed: 209 W ORCHARD ST, Milwaukee, WI - [43.01597451788916, -87.91300366763807]
Parsed: 1559 S 14TH ST, Milwaukee, WI - [43.01469150874257, -87.92938900822297]
Parsed: 1433 W BURNHAM ST, Milwauk

Parsed: 182 E GAUER CR, Milwaukee, WI - [42.98965649321932, -87.90782990896761]
Parsed: 3807 S AUSTIN ST, Milwaukee, WI - [42.97539599999999, -87.90767153043997]
Parsed: 2329 S KINNICKINNIC AV, Milwaukee, WI - [43.002309332765975, -87.90377581101032]
Parsed: 2404 S 5TH ST, Milwaukee, WI - [43.00149536158135, -87.91693203599807]
Parsed: 1708 N 40TH ST, Milwaukee, WI - [43.052526, -87.96358638963613]
Parsed: 2140 N 48TH ST, Milwaukee, WI - [43.05803383236193, -87.97350241906649]
Parsed: 2448 N 29TH ST, Milwaukee, WI - [43.06350541326677, -87.94988536078269]
Parsed: 2473 W MONROE ST, Milwaukee, WI - [43.066308490132265, -87.94447311423035]
Parsed: 2130 N 41ST ST, Milwaukee, WI - [43.058195245628724, -87.96461288631792]
Parsed: 2632 W LLOYD ST, Milwaukee, WI - [43.057809511541194, -87.94735314210284]
Parsed: 4825 W CENTER ST, Milwaukee, WI - [43.06789178122237, -87.97402906060083]
Parsed: 3308 W GALENA ST, Milwaukee, WI - [43.051521456575614, -87.955614]
Parsed: 2759 N 36TH ST, Milwaukee, 

Parsed: 6929 W SILVER SPRING DR, Milwaukee, WI - [43.11928452452563, -87.99831016763807]
Parsed: 5740 N 91ST ST, Milwaukee, WI - [43.122320884817356, -88.02567490074465]
Parsed: 6366 N 76TH ST, Milwaukee, WI - [43.133768245628715, -88.00565485688755]
Parsed: 7608 W GRANTOSA DR, Milwaukee, WI - [43.11054240005059, -88.00638261905922]
Parsed: 8014 W CARMEN AV, Milwaukee, WI - [43.123263289083404, -88.01112213505957]
Parsed: 7736 W KATHRYN AV, Milwaukee, WI - [43.111340453257384, -88.00865257660567]
Parsed: 9316 W APPLETON AV, Milwaukee, WI - [43.114668735789614, -88.02834442143244]
Parsed: 6435 W CARMEN AV, Milwaukee, WI - [43.122989495672165, -87.99196117055226]
Parsed: 1487 N FARWELL AV, Milwaukee, WI - [43.04952495580173, -87.89562189108496]
Parsed: 2033 E KENILWORTH PL, Milwaukee, WI - [43.05843683555322, -87.88478198700756]
Parsed: 1503 N FARWELL AV, Milwaukee, WI - [43.049774607565254, -87.89535681337624]
Parsed: 2625 N DOWNER AV, Milwaukee, WI - [43.06649358090485, -87.87811860257

Parsed: 744 N 4TH ST, Milwaukee, WI - [43.03955055430848, -87.91598725579834]
Parsed: 509 W WISCONSIN AV, Milwaukee, WI - [43.0387804800916, -87.9184609228174]
Parsed: 2735 W STATE ST, Milwaukee, WI - [43.04320051399404, -87.94883275145709]
Parsed: 2028 W CHERRY ST, Milwaukee, WI - [43.050096010746294, -87.93814386236798]
Parsed: 509 W WISCONSIN AV, Milwaukee, WI - [43.0387804800916, -87.9184609228174]
Parsed: 441 N 2ND ST, Milwaukee, WI - [43.035268196851625, -87.91315984812782]
Parsed: 1128 N 20TH ST, Milwaukee, WI - [43.04476724562872, -87.9377944412835]
Parsed: 853 N 23RD ST, Milwaukee, WI - [43.0414022514571, -87.94161057372014]
Parsed: 250 E WISCONSIN AV, Milwaukee, WI - [43.03866951601325, -87.90826283097697]
Parsed: 795 N JEFFERSON ST, Milwaukee, WI - [43.04111845178262, -87.9055601798367]
Parsed: 814 W WISCONSIN AV, Milwaukee, WI - [43.03884949437317, -87.92237291909515]
Parsed: 183 W MICHIGAN ST, Milwaukee, WI - [43.03733509431495, -87.91268747749594]
Parsed: 3017 W HIGHLAND 

...

Parsed: 4061 N 54TH ST, Milwaukee, WI - [43.09110591594984, -87.98070014368938]
Parsed: 7501 W SHERIDAN AV, Milwaukee, WI - [43.117807852328596, -88.0039094705983]
Parsed: 6620 W CAPITOL DR, Milwaukee, WI - [43.090045497691385, -87.99511056217895]
Parsed: 4569 N 65TH ST, Milwaukee, WI - [43.10062283819033, -87.99314113200396]
Parsed: 3925 N 63RD ST, Milwaukee, WI - [43.08868858673321, -87.99082813200395]
Parsed: 5700 W HOPE AV, Milwaukee, WI - [43.09356270649403, -87.98431165846337]
Parsed: 5227 W MELVINA ST, Milwaukee, WI - [43.087946423706875, -87.9786758846811]
Parsed: 5875 N 69TH ST, Milwaukee, WI - [43.12478758673322, -87.99722165753917]
Parsed: 9414 W SHERIDAN AV, Milwaukee, WI - [43.1185044532574, -88.03018358381902]
Parsed: 4146 N 64TH ST, Milwaukee, WI - [43.09289924562873, -87.99192338631792]
Parsed: 6801 W VILLARD AV, Milwaukee, WI - [43.112085539529254, -87.99642058673322]
Parsed: 5405 N 92ND ST, Milwaukee, WI - [43.11604689733378, -88.02724824825461]
Parsed: 8135 W FLORIST

Parsed: 4422 N 80TH ST, Milwaukee, WI - [43.09774182209015, -88.01212662666651]
Parsed: 8238 W APPLETON AV, Milwaukee, WI - [43.10334728088745, -88.01467862389654]
Parsed: 10155 W APPLETON AV, Milwaukee, WI - [43.12220352037069, -88.03898864019827]
Parsed: 6570 N 104TH CT, Milwaukee, WI - [43.13726744160061, -88.04126866261774]
Parsed: 5121 N 105TH ST, Milwaukee, WI - [43.11084600000001, -88.04336015753917]
Parsed: 10303 W FOND DU LAC AV, Milwaukee, WI - [43.139765478137505, -88.04020479929922]
Parsed: 10023 W APPLETON AV #6, Milwaukee, WI - [43.12123234883746, -88.03758675053346]
Parsed: 4243 N 88TH ST, Milwaukee, WI - [43.094367502914196, -88.022579592042]
Parsed: 9045 W BUTLER PL, Milwaukee, WI - [43.10086704411688, -88.02563126937493]
Parsed: 3778 N 82ND ST, Milwaukee, WI - [43.085268450458756, -88.01499541286368]
Parsed: 9120 W HAMPTON AV, Milwaukee, WI - [43.10505351875455, -88.02655225145709]
Parsed: 3868 N 83RD ST, Milwaukee, WI - [43.087689167638075, -88.01622241185314]
Parsed

Parsed: 3615 N HUMBOLDT BL, Milwaukee, WI - [43.08332377990652, -87.89756165032581]
Parsed: 3203 N 2ND ST, Milwaukee, WI - [43.07643025145711, -87.91263165364403]
Parsed: 3524 N 11TH ST, Milwaukee, WI - [43.0822611618097, -87.92473643738836]
Parsed: 1635 W CAPITOL DR, Milwaukee, WI - [43.08942955395597, -87.93188069608748]
Parsed: 2960 N 5TH ST, Milwaukee, WI - [43.07256916763808, -87.91689337189119]
Parsed: 2565 N 5TH ST, Milwaukee, WI - [43.06542358673323, -87.91704011757723]
Parsed: 2900 N 11TH LA, Milwaukee, WI - [43.07147096863639, -87.92527643738836]
Parsed: 1329 W LOCUST ST, Milwaukee, WI - [43.071066023763365, -87.92803509333714]
Parsed: 2104 N MARTIN L KING JR DR, Milwaukee, WI - [43.057868093318646, -87.91412876948877]
Parsed: 2920 N 18TH ST, Milwaukee, WI - [43.07188500000001, -87.93552488631792]
Parsed: 3362 N 22ND ST, Milwaukee, WI - [43.08028205245543, -87.93965491185313]
Parsed: 1917 N 7TH ST, Milwaukee, WI - [43.05504599016999, -87.92011770478754]
Parsed: 3342 N 3RD ST,

Parsed: 2400 W GREENFIELD AV, Milwaukee, WI - [43.0170215115412, -87.94380149999999]
Parsed: 3704 W BURNHAM ST, Milwaukee, WI - [43.01025448600598, -87.960672]
Parsed: 2758 S 35TH ST, Milwaukee, WI - [42.994206040538856, -87.95799490074464]
Parsed: 2526 S 33RD ST, Milwaukee, WI - [42.99897578153841, -87.95546455288621]
Parsed: 1432 S 32ND ST, Milwaukee, WI - [43.016093245628724, -87.95385288963614]
Parsed: 1611 S 30TH ST, Milwaukee, WI - [43.013772419095176, -87.95157004818492]
Parsed: 2730 W PIERCE ST, Milwaukee, WI - [43.024124482110835, -87.94876108673321]
Parsed: 2183 S MUSKEGO AV, Milwaukee, WI - [43.0049835051359, -87.94314161714453]
Parsed: 3100 W PIERCE ST, Milwaukee, WI - [43.02383245715254, -87.95280278864908]
Parsed: 2137 W PIERCE ST, Milwaukee, WI - [43.02418253952927, -87.94062041909515]
Parsed: 1573 S 31ST ST, Milwaukee, WI - [43.014622246044645, -87.95274328785301]
Parsed: 2047 S 30TH ST, Milwaukee, WI - [43.00741014210283, -87.95170807703836]
Parsed: 2010 S 31ST ST, Mil

Parsed: 6336 W KAUL AV, Milwaukee, WI - [43.12811645325738, -87.99053366472387]
Parsed: 8202 W DENVER AV, Milwaukee, WI - [43.14326331504366, -88.01322836138527]
Parsed: 7900 W ACACIA ST, Milwaukee, WI - [43.137831486149274, -88.00924321870852]
Parsed: 6737 N 75TH ST, Milwaukee, WI - [43.13981949362902, -88.0048117728493]
Parsed: 9011 N SWAN RD, Milwaukee, WI - [43.181163349216725, -88.02426218947136]
Parsed: 4928 W VILLARD AV, Milwaukee, WI - [43.111985529863055, -87.97477041618096]
Parsed: 6891 N 60TH ST, Milwaukee, WI - [43.143166948929554, -87.98562920139626]
Parsed: 6300 W CARMEN AV, Milwaukee, WI - [43.123065756392634, -87.98960831601781]
Parsed: 7701 W CALUMET RD, Milwaukee, WI - [43.15598054270419, -88.00662930564329]
Parsed: 6900 N 76TH ST, Milwaukee, WI - [43.14339913627447, -88.00538287016043]
Parsed: 8100 W BROWN DEER RD, Milwaukee, WI - [43.177965461047656, -88.01110366763807]
Parsed: 8524 N SERVITE DR, Milwaukee, WI - [43.17276397530403, -88.00882284728532]
Parsed: 7104 W